# Modelling Motor Imagery BCI with Active Inference

The original paper : 
Simultaneous MRI-EEG during a motor imagery neurofeedback task: an open access brain imaging dataset for multi-modal data integration
Giulia Lioi, Claire Cury, Lorraine Perronnet, Marsel Mano, Elise Bannier, Anatole Lécuyer,  View ORCID ProfileChristian Barillot
doi: https://doi.org/10.1101/862375

First, let's make a quick summing-up of the result of the original paper, then we'll motivate our approach without getting too far into the details of the model implementation.

Finally, we'll propose a few key simulations to enlighten the results of the original task.


## 1. The orignal task

### Noticeable particularities : 
- **Two imaging techniques :** FMRI (fine spatial resolution, but slow dynamics) & EEG (lower spatial resolution, excellent temporal resolution). It was a technical challenge to integrate both measure simultaneously (+ online processing)
- **Two-dimensional feedback** based on those sensor modalities

### The task itself :
- Motor imagery Neurofeedback task (right hand)
- Sensors : 64 channels EEG + fMRI
- Two sub-studies (XP1 and XP2). XP1 (N=10) compared **unimodal** and **bimodal** Neurofeedback. XP2 (N=20) performed **bimodal** NF with a 1D or 2D feedback.
- Participants : right-handed, naive, 18/12,  (mean age 33.7 ± 9.9 years old). Only 26 in the public dataset.

### Experimental paradigms : 

#### XP1. EEG / fMRI / EEG + fMRI

![alt text](resources/xp1_schema.JPG "XP1"){: style="height:320px"}

**Instructions :** (verbally + written at the beginning of each trial) :
> - They were instructed to perform a kinaesthetic motor imagery of the right hand and to find their own strategy to control and bring the ball to the target. 
>
> - They were informed that the NF would be a measure of laterality and that they had to maximize the activity only over the right hand motor region, therefore avoiding to image movements involving the two hands.
>
> - Information about the direction (vertical, horizontal or both) along which the ball could be moved were given


**Trials :** (organized in blocks, 20s rest-20s task)
- 1.  run of motor localization (hand clenching) - 8 blocks
- 2.  run of feedback deprived MI - 5 blocks
- 3.  run of EEG-NF - 10 blocks
- 4.  run of FMRI-NF - 10 block
- 5.  run of EEG/FMRI NF - 10 blocks
- 6.  run of feedback deprived MI - 5 blocks

*Phases 3/4/5 could happen in any order.*

**Biomarkers :**
- fMRI : the left M1 ROI was defined during phase 1 (9x9x3 voxels around the maximum of activation in the motor), and the right M1 ROI was the symmetrical across mid-saggital plane. The resulting laterality index was calculated during training following : $NF_{MRI}=\frac{B_{left}(v)}{B_{left}(rest)} - \frac{B_{right}(v)}{B_{right}(rest)}$, with B the avg of the BOLD signal over the ROI during the trial (v) and at rest. Updated every 2 seconds.
- EEG : EEG power in the 8-12Hz band in electrodes C1+C2 following $NF_{eeg}=\frac{BP_{left}-BP_{right}}{BP_{left}-BP_{right}}$ (Alpha Assymetry Index in the motor area). Updated every 250 ms.

#### XP2. EEG + fMRI | 1D vs 2D feedback

![alt text](resources/xp2_schema.JPG "XP2"){: style="height:320px"}

**Instructions :** (verbally + written at the beginning of each trial) :
> - They were instructed to perform a kinaesthetic motor imagery of the right hand and to find their own strategy to control and bring the ball to the target. 
>
> - They were informed that the NF would be a measure of laterality and that they had to maximize the activity only over the right hand motor region, therefore avoiding to image movements involving the two hands.
>
> - The two NF metaphors (bi-dimensional 2d and unidimensional 1d) were described.


**Trials :** (organized in blocks, 20s rest-20s task)
- 1.  run of feedback deprived MI - 5 blocks
- 2.  3 runs of EEG/FMRI NF with Nmod(subj) - 10 blocks each
- 3.  run of feedback deprived MI - 5 blocks

**Biomarkers :**
- calibration procedure : during phase 1 (*note that this was done during the MIpre phase instead of the motorloc*). EEG data -> Analyzer (Brain Product) -> BCG (ballistocardiogram) correction -> CSP filter to maximize active vs rest. fMRI data -> slice-time correction -> spatial realignment + smoothing -> GLM for activation maps -> ROI are the 9x9x3 voxels around the maximum of activation. No right M1 here.
- fMRI :  The resulting fMRI feedback was calculated during training following : $NF_{MRI}=\frac{B_{ROI}(v)}{B_{ROI}(rest)} - \frac{B_{BG}(v)}{B_{BG}(rest)}$, with $B_{ROI}$ the avg of the BOLD signal over the ROI during the trial (v) and at rest and $B_{BG}$ the BOLD signal in a background deep slice. Updated every 1 second.
- EEG : EEG power in the 8-12Hz band in electrodes C1+C2 following the ERD: $NF_{eeg}=\frac{BP_{left}(rest)-BP_{left}(t)}{BP_{left}(rest)}$ (Event related desynchronization). Updated every 250 ms.


# 2. The model !

We use Active Inference to frame the paradigm in a subject-centric cognitive manner. 

In effect, instead of directly controlling their brain activity, the subjects attempt to regulate its cognition, which leads to changes in the biomarker. Our model attempts to answer a few questions relating to the task : 
- What are the most likely regulated cognitive states ? 
- What is the effect of different temporal scales in the provided feedback ?
- Another ?

But first, we need to constrain our situation a little !